In [2]:
import sqlite3
import pandas as pd
import os

In [4]:
db = os.path.abspath('database.db')

In [6]:
#check the databse tables
with sqlite3.connect(os.path.abspath(db)) as conn:
    cursor = conn.cursor()
    cursor.execute("select name from sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:", tables)
    for table in tables:
        query = f'select * from {table[0]}' 
        df = pd.read_sql_query(query, conn)
        print(df.head())

Tables in the database: []


In [95]:
# # Drop all tables 
with sqlite3.connect(os.path.abspath(db)) as conn:
    cursor = conn.cursor()
    cursor.execute("select name from sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table in tables:
        cursor.execute(f"drop table if exists {table[0]}")
    conn.commit()

In [96]:
#read: https://www.sqlite.org/lang_datefunc.html
with sqlite3.connect(os.path.abspath(db)) as conn:
    cursor = conn.cursor()
    #text 
    cursor.execute('create table if not exists table1(d1 text)')
    cursor.execute("insert into table1(d1) values (datetime('now'))")
    #real 
    cursor.execute("create table if not exists table2(d1 real)")
    cursor.execute("insert into table2(d1) values (julianday('now'))")

    #int 
    cursor.execute('create table if not exists table3(d1 int)')
    cursor.execute("insert into table3(d1) values (strftime('%s','now'))")

    query1 = 'select *, typeof(d1) from table1'
    query2 = 'select *, typeof(d1) from table2'
    query3 = "select *, typeof(d1) from table3"
    #instead of df, use: cursor.execute(query).fetchall()
    df1 = pd.read_sql_query(query1, conn)
    df2 = pd.read_sql_query(query2, conn)
    df3 = pd.read_sql_query(query3, conn)


    

In [108]:
with sqlite3.connect(db) as conn:
    cursor = conn.cursor()


[('2024-10-21 21:53:30', 'text')]

In [97]:
df1

,d1,typeof(d1)
0,2024-10-21 21:53:30,text


In [98]:
df2

,d1,typeof(d1)
0,2.460605e+06,real


In [99]:
df3

,d1,typeof(d1)
0,1729547610,integer


In [104]:
#read: https://www.sqlite.org/lang_datefunc.html
with sqlite3.connect(os.path.abspath(db)) as conn:
    cursor = conn.cursor()
    
    query1 = "select *, case when d1 is not NULL then 'date_exists' else 'no' end 'date_status' from table1"
    df1 = pd.read_sql_query(query1, conn)


In [105]:
df1

,d1,date_status
0,2024-10-21 21:53:30,date_exists
